In [272]:
# Loaded required packages
from collections import Counter
import numpy as np
from collections import defaultdict, deque

# Load input data
file_path = "C:\\Users\\JoeRatterman\\Documents\\GitHub\\2021-advent-of-code\\input-files\\2021-12-16_input.txt"
data_list = open(file_path, 'r').read()

data_list

'9C0141080250320F1802104A08'

In [273]:
# Dictionary of conversion keys
conv_dict = {

    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111'

}

In [274]:
# Function to split string
def str_split(word):
    return list(word)

# Replace list element with dictionary value
def list_replace(list, dictionary):
    return([dictionary.get(item, item) for item in list])

# Function to convert  
def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return(str1) 

# Function to split string every n characters
def split_string_n(string, n):
    return([string[i:i+n] for i in range(0, len(string), n)])

In [275]:
# Function to convert hex code to binary
def hex_to_bin(string):
    return(listToString(list_replace(str_split(data_list), conv_dict)))

# Function to get version from string
def get_version(string):
    return(int(string[0:3], 2))

# Function to get id from string
def get_id(string):
    return(int(string[3:6], 2))

# Function to return len type
def check_len_type(string):
    return(string[6:7])

# Function if len type = 15
def bit_rule_ret(string, bit_rule_calc):
    return(int(string[7:(7 + bit_rule_calc)], 2))

def get_rem_str(string, pk_id):

    if pk_id == 4:
        return(string[6:len(string)])
    else:
        return(string[7:len(string)])

In [276]:
# Function to calculate literal values for version 4 packets
def calc_literal_value(string):

    if len(string) == 5:
        return(int(string[1:], 2))
        
    else:

        lit_val_str = ""

        # Split up the remaining string to bit portions
        lit_val_list = split_string_n(string, 5)

        for i in range(len(lit_val_list)):
            
            lit_val_str += lit_val_list[i][1:]

        if lit_val_str != "":
            return(int(lit_val_str, 2))


In [277]:
# Function to return literal values from remaining string - used at sub-packet stage
def pk_vers_parse(test_string):

    result_dict = {}

    pk_ver = get_version(test_string)
    pk_id = get_id(test_string)
    

    if get_version(test_string) >= 0:

        vals_to_parse = split_string_n(test_string[6:], 5)
        
        packet_end = [x for x in vals_to_parse if x.startswith("0")]
        
        end_index = vals_to_parse.index(packet_end[0]) + 1
       
        literal_val = calc_literal_value(''.join(vals_to_parse[0:end_index]))
       
        remove_string = test_string[0:6] + ''.join(vals_to_parse[0:end_index])

    

        return([pk_id, pk_ver, literal_val, remove_string])

    else:
        return(["", "", "", ""])


In [278]:
# Loop through string to return all literal values remaining
def parse_literal_values(bit_str):

    string_to_parse = bit_str
    result_list = []

    while len(string_to_parse) >= 6:
        
        val = pk_vers_parse(string_to_parse)
        
        result_list.append(val)
        print(string_to_parse)
        string_to_parse = str.removeprefix(string_to_parse, val[3])
        

    return(result_list)


In [279]:
def get_bit_base(string):

    pk_vers = get_version(string)
    pk_id = get_id(string)
    len_type = check_len_type(string)
    bit_rule = 15 if len_type == '0' else 11
    sub_type = bit_rule_ret(string, bit_rule_calc= bit_rule)

    return([pk_vers, pk_id, len_type, bit_rule, sub_type])

In [280]:
# Convert hexadecimal to bit string
bit_string = hex_to_bin(data_list)
bit_string[0:10]

'1001110000'

In [281]:
# Loop through bit string
string_dict_test = {}
bit_list = get_bit_base(bit_string)
string_dict_test[bit_string] = bit_list


while len(bit_string) > 6:

    packet_substring = get_rem_str(bit_string, bit_list[1])[bit_list[3]:]
    

    bit_list = get_bit_base(packet_substring)

    

    if bit_list[1] != 4:

        bit_string = packet_substring
        string_dict_test[packet_substring] = bit_list

    else:
        cntr = 0
        while bit_list[1] == 4:

            if cntr == 0:
                string_dict_test[str(cntr) + "_" + packet_substring] = bit_list

            packet_substring = str.removeprefix(packet_substring, pk_vers_parse(packet_substring)[3])

            bit_list = get_bit_base(packet_substring)
            bit_string = packet_substring
            string_dict_test[str(cntr) + "_" + packet_substring] = bit_list
            cntr += 1
          
    

ValueError: invalid literal for int() with base 2: ''

In [ ]:
value = 0
for i in range(len(string_dict_test.values())):

    value += list(string_dict_test.values())[i][0]


print(value)

889


In [282]:
rev_dict = {}
for k in list(reversed(string_dict_test.keys())):
    
    rev_dict[k] = string_dict_test[k]

#print(rev_dict)

In [283]:
cntr = 0
sub_pack_dict = {}
len_list = []
for k,v in rev_dict.items():
    
    

    if str(k).find("_") > 0:
        string_port = str(k).split("_")[1]
    else:
        string_port = k
    len_str_port = len(string_port)
    len_list.append(len_str_port)


    if v[1] == 4:

        sub_pack_dict[cntr] = pk_vers_parse(str(k).split("_")[1])[2]

    else:

        if v[1] == 0:

            sub_pack_val = sum(list(sub_pack_dict.values())[-num_look_back(v, v[4], len_list[0:-1]):len(list(sub_pack_dict.values()))])
            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val

        if v[1] == 1:

            sub_pack_val = np.prod(list(sub_pack_dict.values())[-num_look_back(v, v[4], len_list[0:-1]):len(list(sub_pack_dict.values()))])
            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val

        if v[1] == 2:

            sub_pack_val = min(list(sub_pack_dict.values())[-num_look_back(v, v[4], len_list[0:-1]):len(list(sub_pack_dict.values()))])
            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val

        if v[1] == 3:

            sub_pack_val = max(list(sub_pack_dict.values())[-num_look_back(v, v[4], len_list[0:-1]):len(list(sub_pack_dict.values()))])
            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val


        if v[1] == 5:

            if list(sub_pack_dict.values())[-2] > list(sub_pack_dict.values())[-1]:

                sub_pack_val = 1
            else:
                sub_pack_val = 0

            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val


        if v[1] == 6:

            if list(sub_pack_dict.values())[-2] < list(sub_pack_dict.values())[-1]:
                
                sub_pack_val = 1
            else:
                sub_pack_val = 0

            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val


        if v[1] == 7:

            if list(sub_pack_dict.values())[-2] == list(sub_pack_dict.values())[-1]:
                
                sub_pack_val = 1
            else:
                sub_pack_val = 0

            #sub_pack_dict = {}
            sub_pack_dict[cntr] = sub_pack_val

    print(k, v)
    print(sub_pack_dict)



    cntr += 1


print(sub_pack_dict)
        

0_0101000001000 [2, 4, '0', 15, 8]
{0: 2}
0_000100000100101000001000 [0, 4, '0', 15, 4738]
{0: 2, 1: 2}
1_110001100000000010000100000100101000001000 [6, 1, '1', 11, 2]
{0: 2, 1: 2, 2: 4}
0_10010000011110001100000000010000100000100101000001000 [4, 4, '0', 15, 7728]
{0: 2, 1: 2, 2: 4, 3: 3}
0_0101000000110010000011110001100000000010000100000100101000001000 [2, 4, '0', 15, 3203]
{0: 2, 1: 2, 2: 4, 3: 3, 4: 1}
0100001000000000100101000000110010000011110001100000000010000100000100101000001000 [2, 0, '1', 11, 2]
{0: 2, 1: 2, 2: 4, 3: 3, 4: 1, 5: 4}
10011100000000010100000100001000000000100101000000110010000011110001100000000010000100000100101000001000 [4, 7, '0', 15, 80]
{0: 2, 1: 2, 2: 4, 3: 3, 4: 1, 5: 4, 6: 0}
{0: 2, 1: 2, 2: 4, 3: 3, 4: 1, 5: 4, 6: 0}


In [285]:
print(v)
print(len_list)
print(len_list[0:8])
print(len_list[7])

[4, 7, '0', 15, 80]
[13, 24, 42, 53, 64, 82, 104]
[13, 24, 42, 53, 64, 82, 104]


IndexError: list index out of range

In [222]:
def num_look_back(list_obj, str_len, prev_lens_list):


    if list_obj[3] == 11:
        look_back = list_obj[4]
        
    else:
        

        val = [x for x in prev_lens_list if prev_lens_list[-1] - x == str_len][0]
        look_back_index = prev_lens_list.index(val)
        look_back = len(prev_lens_list) - look_back_index

    return(look_back)